In [ ]:
import numpy as np
from suite2p.io import BinaryFile
import matplotlib.pyplot as plt
from pathlib import Path
from cellpose import transforms

root = Path('/media/carsen/disk2/suite2p_paper/GT1')
db = np.load(root / "suite2p/plane0/db.npy", allow_pickle=True).item()
settings = np.load(root / "suite2p/plane0/settings.npy", allow_pickle=True).item()

reg_outputs = np.load(root / "suite2p/plane0/reg_outputs.npy", allow_pickle=True).item()
Ly, Lx = db["Ly"], db["Lx"]

with BinaryFile(Ly, Lx, filename=root / "suite2p/plane0/data_raw.bin", write=False) as f:
    frand = np.array(f[::50])

fig = plt.figure(figsize=(14,7), dpi=300)
grid = plt.GridSpec(3, 7, wspace=0.25, hspace=0.45, figure=fig, 
                        bottom=0.04, top=0.93, left=0.02, right=0.98)

frands = frand[:, :Ly//2, :Ly//2].copy()
fmin, fmax = np.percentile(frands, 0.1), np.percentile(frands, 98)
frands = np.clip((frands.astype("float32") - fmin) / (fmax - fmin), 0, 1)    

Ly0, Lx0 = frands.shape[1:]
ly = Ly0 - 10
lx = Lx0 - 10
by = 40
#iex = np.arange(0, len(frand), 10)
iex = [0, 15, 28, 30, 50, 60, 80,100]
n = len(iex)
raw_ex = frands[iex, 10:10+ly, 10:10+lx]

i0 = (np.abs(np.diff(reg_outputs["xoff"])) + np.abs(np.diff(reg_outputs["yoff"]))).argsort()[::-1][0] 
i1 = i0 + 1
#i0 = reg_outputs["xoff"].argmin()
#i1 = reg_outputs["xoff"].argmax()

with BinaryFile(Ly, Lx, filename=root / "suite2p/plane0/data_raw.bin", write=False) as f:
    frand = f[[i0, i1]].copy()

with BinaryFile(Ly, Lx, filename=root / "suite2p/plane0/data.bin", write=False) as f:
    frand_reg = f[[i0, i1]].copy()

frands = np.concatenate((frand, frand_reg), axis=0)

fmin, fmax = np.percentile(frands, 5), np.percentile(frands, 99.9)
frands = np.clip((frands.astype("float32") - fmin) / (fmax - fmin), 0, 1)    
reg_ex = frands.reshape(2, 2, Ly, Lx)
print(frands.shape)
Ly, Lx = frands.shape[1:]

with BinaryFile(Ly, Lx, filename=root / "suite2p/plane0/data.bin", write=False) as f:
    frand = np.array(f[::50])


stat = np.load(root / "suite2p/plane0/stat.npy", allow_pickle=True)
iscell = np.load(root / "suite2p/plane0/iscell.npy", allow_pickle=True)
F = np.load(root / "suite2p/plane0/F.npy", allow_pickle=True)
Fneu = np.load(root / "suite2p/plane0/Fneu.npy", allow_pickle=True)

    
max_proj = frand.max(axis=0)

masks = np.zeros((Ly, Lx), 'uint16')
np.random.seed(0)
iperm = np.random.permutation(len(stat))
#iperm = np.arange(0, len(stat))
iscell0 = np.zeros(len(stat))
for i in range(len(stat)):
    ypix, xpix = stat[i]['ypix'], stat[i]['xpix']
    overlap = stat[i]['overlap']
    ypix, xpix = ypix[~overlap], xpix[~overlap]
    masks[ypix,xpix] = iperm[i]+1
    iscell0[iperm[i]] = iscell[i, 1]
colors = plt.get_cmap('hsv')(np.linspace(0, 1, len(stat)))[iperm]

np.save('results/pipeline_ex.npy', {'raw_ex': raw_ex, 'reg_ex': reg_ex, 'max_proj': max_proj,
                                    'stat': stat, 'iscell0': iscell0, 
                                    'iperm': iperm, 'colors': colors, 
                                    'masks': masks, 'F': F, 'Fneu': Fneu})

In [ ]:
dat = np.load('results/pipeline_ex.npy', allow_pickle=True).item()
dat2 = np.load('results/manycells.npy', allow_pickle=True).item()

In [ ]:
from fig_utils import * 
import figures 
import importlib 
importlib.reload(figures)

gui_img = plt.imread('figures/gui.png')# _cells.png')
planes_img = plt.imread('figures/planes.png')

fig = figures.pipeline_fig(gui_img, planes_img, **dat, **dat2)
fig.savefig('figures/fig1.pdf', dpi=150)

In [ ]:
importlib.reload(figures)
dat = np.load('results/manycells.npy', allow_pickle=True).item()
fig = figures.suppfig_planes(**dat)
    
fig.savefig('figures/suppfig_planes.pdf', dpi=150)